From https://datascience.stackexchange.com/questions/55566/tool-for-labeling-audio

In [ ]:
from pathlib import Path
import pandas as pd
import librosa
import librosa.display
import numpy as np
from tqdm import tqdm

dataset_path = Path('datasets')


In [ ]:
# build dataset pandas df
featuresdf = pd.read_pickle('data_chunks_df.pickle')

print('Finished feature extraction from ', len(featuresdf), ' files')

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [ ]:
from simple_model import create_model
model = create_model()

In [ ]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy) 

In [ ]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 
saved_folder = 'model_saves'
if not os.path.isdir(saved_folder):
    os.mkdir(saved_folder)

num_epochs = 1000
num_batch_size = 256

checkpointer = ModelCheckpoint(filepath=f'{saved_folder}/weights.best.basic_cnn.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

In [ ]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])